# Notes on Python
## Examples for reference, tips, Best Practices

Based on the Course: Core Python (Custom Attributes and Descriptors) at PluralSight

Author: Gonçalo Felício  
Date: 04/2022  
Provided by: ISIWAY

Something like a pocketbook to come to for quick references, examples, and tips of best practices, compiled with my own preferences  
Loosely divided by subject, and with some degree, by the respective modules

Objects store attributes in an internal dictionary `__dict__`, that maps attribute names to attribute values  
`__dict__` can be directly manipulated to create, retrieve, update and delete attributes. We can also access the dictionary via the `vars()` function    
Any failure to retrieve an item by normal means causes the `__getattr__` to be invoked, if it has been defined  
Assignment and deletion of attributes can be costumized by overriding `__setattr__` and `__delattr__`  
Calls to `__hasattr__` can also invoke `__getattr__` which can lead to undesired infinite recursive loop  
In general its better to use `__getattr__` but ocasionally, we may need to intercept all attributes access with `__getattribute__`  
Classes have their own `__dict__` which stores method callables  
`__slots__` are a way to make objects more memory efficient, in detriment of dynamism and flexibility  

Tip: Prefer composition over Inheritance. This means that, sometimes, when handling related classes, it's better for a class to contain another, rather than inheriting from another, as this may lead to awkward and fragile implementations

In [60]:
class Vector:
    
    def __init__(self, **components):
        private_components = {f'_{k}': v for k, v in components.items()}
        self.__dict__.update(private_components)
        
    def __getattr__(self, name):
        private_name = f'_{name}'
        try:
            return self.__dict__[private_name]
        except KeyError:
            raise AttributeError(f'{self} object has no attribute {name!r}')
    
    def __setattr__(self, name, value):
        raise AttributeError(f"Can't set attributes in object {self!r}")

    def __delattr__(self, name):
        raise AttributeError(f"Can't delete attributes in object {self!r}")
        
    def __repr__(self):
        return '{}({})'.format(
            type(self).__name__,
            ", ".join('{k}={v}'.format(
                        k=k[1:],
                        v=v
                        ) for k, v in vars(self).items() # the other way to access __dict__
            )
        )

In [61]:
v = Vector(x=1, y=3, z=5)

In [62]:
v.t # attributes that don't exist raise appropriate error (delegating from __getattr__)

AttributeError: Vector(x=1, y=3, z=5) object has no attribute 't'

In [63]:
v.x # properly gets attribute value

1

In [64]:
v._x # but in fact is a fake, as real attribute name is _x

1

In [65]:
v.x = 2 # can't set new values, its been implemented as immutable

AttributeError: Can't set attributes in object Vector(x=1, y=3, z=5)

In [66]:
del v.x # can't delete new values, its been implemented as immutable

AttributeError: Can't delete attributes in object Vector(x=1, y=3, z=5)

In [67]:
delattr(v, 'x')

AttributeError: Can't delete attributes in object Vector(x=1, y=3, z=5)

In [68]:
repr(v)

'Vector(x=1, y=3, z=5)'

The `property` function is usually used as a decorator, but can also be called normally by passing *getter* and *setter* functions to `property`, returning a `descriptor`  
The `descriptor` protocol comprises of `__get__`, `__set__` and `__delete__`  
The `descriptor` objects are shared by all owning instances, this can be achieved by using a *WeakKeyDictionary*  
The `descriptor` can also be retrieved by the owning class, instead via the instance, in this case the *instance* argument of `__get__` is set to *None*  
Implement `__set_name__` to retrive the descriptors attribute name  

In [147]:
from weakref import WeakKeyDictionary

class Positive:
    """A data descriptor for positive check of numbers"""
    
    def __init__(self):
        self._instance_data = WeakKeyDictionary()
        
    def __set_name__(self, owner, name):
        self._name = name
    
    def __get__(self, instance, owner):
        if instance is None: #check wether the descriptor is being retrived from the Class
            return self
        return self._instance_data[instance]
    
    def __set__(self, instance, value):
        if value <=0:
            raise ValueError(f'Value {value} is not positive.')
        self._instance_data[instance] = value
        
    def __delete__(self, instance):
        raise AttributeError(f"Can't delete attributes.")

In [148]:
class Planet:
    def __init__(self, name, radius_meters, mass_kilograms):
        self.name = name
        self.radius_meters = radius_meters
        self.mass_kilograms = mass_kilograms
    
    # using normal property to set attribute name
    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        if not value:
            raise ValueError('Cannot set empty name.')
        self._name = value
    
    # using descriptor Positive to set and check the numeric attributes
    radius_meters = Positive()
    mass_kilograms = Positive()

In [149]:
Planet.radius_meters # Positive is the descriptor object of the class attribute

In [150]:
pluto = Planet(name='pluto', radius_meters=123456, mass_kilograms=-4)

ValueError: Value -4 is not positive.

In [164]:
pluto = Planet(name='pluto', radius_meters=123456, mass_kilograms=4)

In [165]:
pluto.radius_meters

123456

In [166]:
pluto.radius_meters = 654321
pluto.radius_meters

654321

In [154]:
del pluto.mass_kilograms

AttributeError: Can't delete attributes.

In [167]:
mercury = pluto = Planet(name='Mercury', radius_meters=78910, mass_kilograms=8)

In [168]:
mercury.radius_meters

78910

In [176]:
# with the weak ref dictionary, we see that the descriptor objects really are shared between instances
pprint(Planet.mass_kilograms._instance_data) # this is possible because of the __set_name__ implementation
pprint(dict(Planet.mass_kilograms._instance_data))

{<__main__.Planet object at 0x0000021418FB3E80>: 4,
 <__main__.Planet object at 0x0000021418FCB3A0>: 8}


In [178]:
pprint(Planet.__dict__)

mappingproxy({'__dict__': <attribute '__dict__' of 'Planet' objects>,
              '__doc__': None,
              '__init__': <function Planet.__init__ at 0x0000021418F7BF70>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Planet' objects>,
              'mass_kilograms': <__main__.Positive object at 0x0000021418F73070>,
              'name': <property object at 0x0000021418F74770>,
              'radius_meters': <__main__.Positive object at 0x0000021418F43E20>})


Data and non-Data descriptors have different lookup precedences  
In Data descriptors, class attributes take precedence when an instance attribute with the same name exists  
In non-Data descriptos the oposite occurs